In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_excel(r"C:\Users\u22v09\desktop\book1.xlsx")
df['time'] = pd.to_datetime(df['time'])
#df=df[df['EM_id']==1]  ################# EM_id=1
df = df[(df['time'] > '2020-05-02 00:00:00.000000000') & (df['time'] < '2020-05-03 00:00:00.000000000')]

print(df.head())

                           time  EM_Current Avg (A)  EM_Current Ph1 (A)  \
0 2020-05-02 04:45:24.826769920                 0.0                 0.0   
1 2020-05-02 04:45:24.904900096                 1.5                 4.7   
2 2020-05-02 04:45:25.248640000                15.7                26.9   
3 2020-05-02 04:45:25.264270080                 0.0                 0.0   
4 2020-05-02 04:45:25.436140032                 0.0                 0.0   

   EM_Voltage Ph1-N (V)  mean_voltage  EM_id  
0                   0.0      0.000000    1.0  
1                 230.1    231.266667    4.0  
2                 231.1    231.333333    5.0  
3                 231.1    231.200000   11.0  
4                 231.1    231.200000    7.0  


In [14]:
max_time= df.loc[df["EM_Voltage Ph1-N (V)"]==df["EM_Voltage Ph1-N (V)"].max()]['time'].max()
max_time

Timestamp('2020-05-02 12:30:33.068349952')

In [20]:
class agg():

    def __init__(self, Upper_threshold,lower_threshold):
        self.Upper_threshold = Upper_threshold
        self.lower_threshold = lower_threshold
        
    def set_val(self,row):
        if row['EM_Voltage Ph1-N (V)'] > self.Upper_threshold :
            return "high"
        elif row['EM_Voltage Ph1-N (V)'] < self.lower_threshold :
            return "low"
        else:
            return "medium"

    def daily(self,df1):
        df0 = df1[(df1['EM_Voltage Ph1-N (V)'] == 0)]
        df1 = df1[(df1['EM_Voltage Ph1-N (V)'] > 0)]
        
        Date = df1['time'].dt.date.min()

        MEAN=df1['EM_Voltage Ph1-N (V)'].mean()
        MAX=df1['EM_Voltage Ph1-N (V)'].max()
        MIN=df1['EM_Voltage Ph1-N (V)'].min()
        quantile25=df1['EM_Voltage Ph1-N (V)'].quantile(0.25)
        quantile75=df1['EM_Voltage Ph1-N (V)'].quantile(0.75)
        IQR = quantile75-quantile25
        SD = df1['EM_Voltage Ph1-N (V)'].std()
        
        df1 = df1.assign(flag2=df.apply(self.set_val, axis=1))
        y=pd.DataFrame(df1.groupby([(df1.flag2 != df1.flag2.shift()).cumsum()])['time'].apply(lambda y:(y.iloc[-1]-y.iloc[0]).total_seconds()/60))
        y['flag2']=df1.loc[df1.flag2.shift(-1) != df1.flag2]['flag2'].values
        y.reset_index(drop=True,inplace=True)
        
        duration_high=y[y['flag2']=="high"].time.sum()
        duration_medium=y[y['flag2']=="medium"].time.sum()
        duration_low=y[y['flag2']=="low"].time.sum()
             
        duration_0=df0.groupby(df0.index.to_series().diff().ne(1).cumsum())['time'].agg(lambda x: abs((x.iloc[-1] - x.iloc[0]).total_seconds()/60)).sum()

        non_zeros=len(df1)
        zeros=len(df0)
        overlimit_count=len(df1[df1['flag2']=="high"])
        underlimit_count=len(df1[df1['flag2']=="low"])
        onlimit_count=len(df1[df1['flag2']=="medium"])
        
        
        df1['voltage_difference']=df['EM_Voltage Ph1-N (V)'].diff(-1)
        Average_jump=abs(df1['voltage_difference']).mean()
        min_jump=abs(df1['voltage_difference']).min()
        max_jump=abs(df1['voltage_difference']).max()
        df1['jumps_instantaneous']= np.where(abs(df1["voltage_difference"])> (Average_jump*1.1),1,0)
        jumps_instantaneous=len(df1[df1['jumps_instantaneous']==1])
        
        df1['delta_volt'] = df1['EM_Voltage Ph1-N (V)']- MEAN
        df1['jumps_average']= np.where(abs(df1["voltage_difference"])> (abs(df1['delta_volt']).mean()*1.1),1,0)
        jumps_average=len(df1[df1['jumps_average']==1])
        
        
        upper_difference=MEAN-self.Upper_threshold
        lower_difference=MEAN-self.lower_threshold

        A=[Date,MEAN,MAX,MIN,quantile25,quantile75,IQR,SD,upper_difference,lower_difference,duration_high,duration_medium,duration_low,duration_0,non_zeros,zeros,overlimit_count,underlimit_count,onlimit_count,Average_jump,min_jump,max_jump,jumps_instantaneous,jumps_average]
        FINAL = pd.DataFrame(columns=['Date','MEAN','MAX','MIN','quantile25','quantile75','IQR','SD','upper_difference','lower_difference','duration_high','duration_medium','duration_low','duration_0','non_zeros','zeros','overlimit_count','underlimit_count','onlimit_count','Average_jump','min_jump','max_jump','jumps_instantaneous','jumps_average'],data=np.array(A).reshape(-1,len(A)))
      
        return FINAL
    
    def sensor(self,df):
        final_sen=df.groupby(df['EM_id']).apply(self.daily)
        final_sen.reset_index(inplace=True)
        return final_sen
        

In [21]:
agg1 = agg(245,190)
agg1.sensor(df[['time','EM_Voltage Ph1-N (V)','EM_id']])

,EM_id,level_1,Date,MEAN,MAX,MIN,quantile25,quantile75,IQR,SD,...,non_zeros,zeros,overlimit_count,underlimit_count,onlimit_count,Average_jump,min_jump,max_jump,jumps_instantaneous,jumps_average
0,1.0,0,2020-05-02,247.88,256.6,240,244.1,251.9,7.8,4.35072,...,20281,10282,12466,0,7815,17.1587,0,28.5,8259,20237
1,4.0,0,2020-05-02,230.115,231.3,227.1,230.1,230.1,0,0.563136,...,24618,0,0,0,24618,26.3132,0,231.1,2385,19183
2,5.0,0,2020-05-02,230.893,237,228,230.3,231.3,1,1.00231,...,24116,0,0,0,24116,25.6353,0,236.8,2319,11423
3,7.0,0,2020-05-02,230.801,241.1,228,230.3,231.3,1,0.805658,...,27408,0,0,0,27408,27.8193,0,231.3,2869,13145
4,11.0,0,2020-05-02,230.758,231.4,228,230.3,231.3,1,0.812487,...,26739,0,0,0,26739,26.7787,0,231.3,2685,12617


In [52]:

df[['time','EM_Voltage Ph1-N (V)','EM_id']].groupby(df['EM_id']).apply(agg1.daily)


,,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
EM_id,,,,,,,,,,,,,,,,,,,,,,
1.0,0,2020-05-02,1,247.88,256.6,240,244.1,251.9,7.8,4.35072,2.88039,...,20281,10282,12466,0,7815,17.1587,0,28.5,8259,20237
4.0,0,2020-05-02,4,230.115,231.3,227.1,230.1,230.1,0,0.563136,-14.8852,...,24618,0,0,0,24618,26.3132,0,231.1,2385,19183
5.0,0,2020-05-02,5,230.893,237,228,230.3,231.3,1,1.00231,-14.1071,...,24116,0,0,0,24116,25.6353,0,236.8,2319,11423
7.0,0,2020-05-02,7,230.801,241.1,228,230.3,231.3,1,0.805658,-14.1993,...,27408,0,0,0,27408,27.8193,0,231.3,2869,13145
11.0,0,2020-05-02,11,230.758,231.4,228,230.3,231.3,1,0.812487,-14.242,...,26739,0,0,0,26739,26.7787,0,231.3,2685,12617


In [ ]:
['Date','MEAN','MAX','MIN','quantile25','quantile75','IQR','SD','upper_difference','lower_difference','duration_high','duration_medium','duration_low','duration_0','non_zeros','zeros','overlimit_count','underlimit_count','onlimit_count','Average_jump','min_jump','max_jump','jumps_instantaneous','jumps_average']


In [ ]:
class table():

    def __init__(self, Upper_threshold,lower_threshold):
        self.Upper_threshold = Upper_threshold
        self.lower_threshold = lower_threshold
        
    def categorization(self,df):
        bins = [0, self.lower_threshold, self.Upper_threshold, np.inf]
        names = ['low', 'medium', 'high']
        d = dict(enumerate(names, 1))
        df['Status'] = np.vectorize(d.get)(np.digitize(df['PF'], bins))
        return df
    
    def percentile(self,n):
        def percentile_(x):
            return np.percentile(x, n)
        percentile_.__name__ = 'Q_%s' % n
        return percentile_
    
    def minimum(self,df):
        df=self.categorization(df)
        x=df.groupby(['EM_id'])['EM_Voltage Ph1-N (V)].agg({'Minimum':'min','Maximum':'max','Average':'mean' })
        x.reset_index(inplace=True)
        return x
                                
    def percentile_dict(self)
        key=['min','Q_01','Q_05','Q_25','Q_50','mean','Q_75','Q_90','Q_95','Q_99','max']
        values=['min',self.percentile(1),self.percentile(5),self.percentile(25),self.percentile(50),'mean',self.percentile(75),self.percentile(90),self.percentile(95),self.percentile(99),'max']
        summary_dict=dict(zip(key,values)) 

    final_df=df.groupby('mt_id').agg({'time':'count','vehicle_speed':summary_dict_veh_speed,'accel':summary_dict, 'a_v_feature':summary_dict,'engine_speed':summary_dict,'distance':lambda x:x.iloc[-1]-x.iloc[0],
                           'flag_accel':self.duration,'flag_decel':self.duration,'flag_cruise':self.duration,'flag_creeping':self.duration,'vflag_0_20':self.duration,'vflag_20_40':self.duration,'vflag_40_60':self.duration,'vflag_60':self.duration,'flag_idle':self.duration})

        

In [ ]:
 class yoo():
        
    def percentile(self,n):
        def percentile_(x):
            return np.percentile(x, n)
        percentile_.__name__ = 'Q_%s' % n
        return percentile_

In [ ]:
test=yoo()
test.percentile(25)